# Optimizacija rojem čestica (Particle Swarm Optimization, PSO)

Optimizacije rojem čestica je primer algoritma iz oblasti inteligencija grupa (swarm intelligence).
Ovi algoritmi su obično inspirisani prirodom i počivaju na sledećoj ideji.
Svaka pojedinačna jedinka je jednostavna, ali čitava grupa kolektivno se ponaša na neki način "inteligentno".

Kod PSO algoritma jedinke traže minimum funkcije kretanjem kroz prostor $R^n$.
Pritom svaka jedinka zna najbolje rešenje do koga je stigla, kao i najbolje rešenje do kog je čitav roj stigao.
Brzina, tj. vektor pomeraja čestice se u svakoj iteraciji računa sabiranjem tri komponente:
- inercija, odnosno prethodna brzina,
- kognitivna brzina - vektor od trenutne pozicije čestice do pozicije najboljeg rešenja do kog je ona stigla,
- socijalna brzina - vektor od trenutne pozicije čestice do pozicije najboljeg rešenja do kog je čitav roj stigao.

Parametrima $c_{inertia}$, $c_{social}$ i $c_{cognitive}$ množimo inerciju, socijalnu, odnosno kognitivnu brzinu i na taj način kontrolišemo šta nam je od ta tri bitnije.
U toku rada algoritma bismo mogli i da menjamo vrednosti ovih parametra. Da li biste smanjivali ili povećavali vrednost $c_{social}$? A $c_{cognitive}$?
Šta bi se desilo ako bi vrednost $c_{social}$ bila nula?
A šta ako bi $c_{cognitive}$ bio nula?

In [2]:
import numpy as np

[Rastriginova funkcija](https://en.wikipedia.org/wiki/Rastrigin_function) je primer neprekidne funkcije kod koje je teško pronaći globalni minimum usled postojanja velikog broja lokalnih minimuma.

Globalni minimum se nalazi u nuli i vrednost funkcije u toj tački je nula.

In [3]:
def rastrigin(x: np.ndarray) -> float:
    A = 10
    n = len(x)
    return A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n))

In [8]:
num_dimensions = 2
bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])

In [27]:
class Particle:
    swarm_best_position = None
    swarm_best_value = float('inf')

    def __init__(self, bounds: np.ndarray, f: callable, c_inertia: float, c_social: float, c_cognitive: float):
        self.c_inertia = c_inertia
        self.c_social = c_social
        self.c_cognitive = c_cognitive
        self.f = f

        self.lower_bounds = bounds[:,0]
        self.upper_bounds = bounds[:,1]

        self.position = self.initialize_position()
        self.velocity = self.initialize_velocity(len(bounds))

        self.value = f(self.position)

        self.personal_best_position = self.position.copy()
        self.personal_best_value = self.value

        if Particle.swarm_best_position is None or Particle.swarm_best_value > self.value:
            Particle.swarm_best_value = self.value
            Particle.swarm_best_position = self.position.copy()

    def initialize_position(self) -> np.ndarray:
        return np.random.uniform(self.lower_bounds, self.upper_bounds)
    
    def initialize_velocity(self, num_dimensions: int):
        return np.random.uniform(-1, 1, size=num_dimensions)
    
    def update_velocity(self):
        r_s = np.random.random(len(self.position))
        r_c = np.random.random(len(self.position))
        assert self.swarm_best_position is not None
        social_velocity = self.c_social * r_s * (self.swarm_best_position - self.position)
        cognitive_velocity = self.c_cognitive * r_c * (self.personal_best_position - self.position)
        self.velocity = self.c_inertia * self.velocity + social_velocity + cognitive_velocity

    def move(self):
        self.update_velocity()
        self.position = np.clip(self.position + self.velocity, self.lower_bounds, self.upper_bounds)
        self.value = self.f(self.position)
        if self.value < self.personal_best_value:
            self.personal_best_position = self.position.copy()
            self.personal_best_value = self.value
            if self.value < Particle.swarm_best_value:
                Particle.swarm_best_position = self.position.copy()
                Particle.swarm_best_value = self.value

In [28]:
def pso(num_iters: int, swarm_size: int, bounds: np.ndarray, f: callable, c_inertia: float, c_social: float, c_cognitive: float):
    swarm = [Particle(bounds, f, c_inertia, c_social, c_cognitive) for _ in range(swarm_size)]
    for _ in range(num_iters):
        for p in swarm:
            p.move()
    return Particle.swarm_best_position, Particle.swarm_best_value

In [29]:
pso(swarm_size=20, num_iters=100, bounds=bounds, f=rastrigin, c_inertia=0.5, c_social=1, c_cognitive=1)

(array([-3.40731713e-09, -4.69669696e-10]), np.float64(0.0))